### Project Scenario

A multi-national firm has hired you as a data engineer. Your job is to access and process data as per requirements.

Your boss asked you to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, you need to transform the data and store it in USD, GBP, EUR, and INR per the exchange rate information made available to you as a CSV file. You should save the processed information table locally in a CSV format and as a database table. Managers from different countries will query the database table to extract the list and note the market capitalization value in their own currency.

### Directions

1. Write a function to extract the tabular information from the given URL under the heading By Market Capitalization, and save it to a data frame.
2. Write a function to transform the data frame by adding columns for Market Capitalization in GBP, EUR, and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
3. Write a function to load the transformed data frame to an output CSV file.
4. Write a function to load the transformed data frame to an SQL database server as a table.
5. Write a function to run queries on the database table.
6. Run the following queries on the database table:
   - Extract the information for the London office, that is Name and MC_GBP_Billion
   - Extract the information for the Berlin office, that is Name and MC_EUR_Billion
   - Extract the information for New Delhi office, that is Name and MC_INR_Billion
7. Write a function to log the progress of the code.
8. While executing the data initialization commands and function calls, maintain appropriate log entries.


In [ ]:
%pip install pandas
%pip install numpy
%pip install bs4

In [90]:
import sqlite3
import requests
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime

from typing import List

In [5]:
banks_url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'

rates_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'

table_attribs_initial = ['Name', 'MC_USD_Billion']
table_attribs_final = [
    'Name',
    'MC_USD_Billion',
    'MC_GBP_Billion',
    'MC_EUR_Billion',
    'MC_INR_Billion'
]

db_name = 'Banks.db'
table_name = 'Largest_banks'
log_file = './code_log.txt'
out_csv = './Largest_banks_data.csv'

In [3]:
def log_progress(message: str, filename: str = './code_log.txt') -> None:
    # Year-Monthname-Day-Hour-Minute-Second
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()  # get current timestamp
    timestamp = now.strftime(timestamp_format)

    with open(filename, "a") as f:
        f.write(timestamp + ' : ' + message + '\n')


log_progress('Starting program...')

In [91]:
def extract(url: str, table_attribs: List[str]) -> pd.DataFrame:
    ''' 
    This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. 
    '''

    html_page = requests.get(url).text
    data = BeautifulSoup(html_page, 'html.parser')

    df = pd.DataFrame(columns=table_attribs)

    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')

    for row in rows:
        col = row.find_all('td')

        if len(col) != 0:
            if col[1].find_all('a') is not None:
                data_dict = {
                    'Name': str(col[1].find_all('a')[1]['title']),
                    'MC_USD_Billion': float(col[2].contents[0][:-1])
                }

                temp_df = pd.DataFrame(data_dict, index=[0])

                '''
                This line was added because for some reason I can't figure out, I keep getting
                this warning from pandas:

                FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries
                is deprecated. In a future version, this will no longer exclude empty or all-NA 
                columns when determining the result dtypes. To retain the old behavior, exclude 
                the relevant entries before the concat operation.
                '''
                # Dropping columns that are all NA
                df = df.dropna(axis=1, how='all')

                df = pd.concat([df, temp_df], ignore_index=True)

    return df

In [92]:
data = extract(banks_url, table_attribs_initial)

In [ ]:
data

In [86]:
def transform(df: pd.DataFrame, csv_path: str) -> pd.DataFrame:
    ''' 
    This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies
    '''
    exchange_rate = pd.read_csv(csv_path)
    exchange_rate = exchange_rate.set_index('Currency')['Rate'].to_dict()

    df['MC_GBP_Billion'] = [
        np.round(x * exchange_rate['GBP'], 2) for x in df['MC_USD_Billion']
    ]

    df['MC_EUR_Billion'] = [
        np.round(x * exchange_rate['EUR'], 2) for x in df['MC_USD_Billion']
    ]

    df['MC_INR_Billion'] = [
        np.round(x * exchange_rate['INR'], 2) for x in df['MC_USD_Billion']
    ]

    return df

In [87]:
transformed_data = transform(data, rates_url)

In [88]:
transformed_data

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,Bank of America,231.52,185.22,215.31,19204.58
2,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,HDFC Bank,157.91,126.33,146.86,13098.63
5,Wells Fargo,155.87,124.70,144.96,12929.42
6,HSBC,148.90,119.12,138.48,12351.26
7,Morgan Stanley,140.83,112.66,130.97,11681.85
8,China Construction Bank,139.82,111.86,130.03,11598.07
9,Bank of China,136.81,109.45,127.23,11348.39


In [89]:
transformed_data['MC_EUR_Billion'][4]

146.86

In [96]:
def load_to_csv(df: pd.DataFrame, output_path: str) -> None:
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path)

In [97]:
load_to_csv(transformed_data, out_csv)

In [107]:
sql_connection = sqlite3.connect(db_name)

In [99]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

In [100]:
load_to_db(transformed_data, sql_connection, table_name)

In [110]:
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    query_result = pd.read_sql(query_statement, sql_connection)
    print(query_result)

In [113]:
all_banks_query = 'SELECT * FROM Largest_banks'

run_query(all_banks_query, sql_connection)

                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

In [103]:
# Average market capitalization of all banks in Billion USD
avg_mc_query = 'SELECT AVG(MC_GBP_Billion) FROM Largest_banks'
run_query(avg_mc_query, sql_connection)

SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0              151.987


In [104]:
# Print only the names of the top 5 banks
top_5_query = 'SELECT Name from Largest_banks LIMIT 5'

run_query(top_5_query, sql_connection)

SELECT Name from Largest_banks LIMIT 5
                                      Name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank


In [105]:
sql_connection.close()